# Run Joint Learning Benchmark

In [1]:
%cd ..

/home/adam/Projects/hybrid-transformer


In [2]:
import os
import torch
import wandb

from hybrid_transformer.configs.task import TaskConfig
from hybrid_transformer.configs.model import ModelConfig
from hybrid_transformer.configs.trainer import TrainerConfig
from hybrid_transformer.configs.logger import LoggerConfig

from hybrid_transformer.utils.datasets.auto import AutoDataset
from hybrid_transformer.utils.tokenizers.auto import AutoTokenizer
from hybrid_transformer.models.auto import AutoModel
from hybrid_transformer.utils.loggers.wandb import WandbLogger

from hybrid_transformer.trainers.trainer import Trainer

from scripts.pretrain.train import DEFAULT_CONFIG_FILES

from hybrid_transformer.utils.objectives.guacamol.objective import GUACAMOL_TASKS
from hybrid_transformer.models.prediction import PREDICTION_MODEL_CONFIGS
%load_ext autoreload
%autoreload 2

/home/adam/miniconda3/envs/hybrid-transformer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-30 22:05:39.050616: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-30 22:05:39.083278: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 22:05:39.235551: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 22:05:39.235585: E external/local_xla/xla/stream_executor/cu

In [6]:
# Load configs

task_config_path = lambda: f'./configs/tasks/guacamol/{guacamol_task}/config.json'


for guacamol_task in GUACAMOL_TASKS:
    
    task_config = TaskConfig.from_pretrained(task_config_path())
    task_config.validate = True
    task_config.split = 'val'
    dataset = AutoDataset.from_config(task_config)
    print(f"Loaded {task_config.target_label} data.")
    
    for model_name, path_to_model_config in PREDICTION_MODEL_CONFIGS.items():
        
        model_config = ModelConfig.from_pretrained(path_to_model_config)

        trainer_config = TrainerConfig.from_pretrained('./configs/trainers/debug/')
        logger_config = LoggerConfig.from_pretrained(DEFAULT_CONFIG_FILES['logger'])
        
        out_dir = f'./results/regression_task/guacamol/{model_name}/{guacamol_task}'
        trainer_config.out_dir = out_dir
        logger_config.name = model_name + '_' + guacamol_task
        logger_config.project = 'debug'
        trainer_config.enable_save_checkpoint = False
        task_config.validate = False
        logger_config.wandb_log = True
        
        dataset = AutoDataset.from_config(task_config)
        tokenizer = AutoTokenizer.from_config(task_config)                
        model = AutoModel.from_config(model_config)
        logger = WandbLogger(logger_config, [task_config, model_config, trainer_config])
        trainer = Trainer(config=trainer_config, model=model, train_dataset=dataset, eval_dataset=dataset, tokenizer=tokenizer, logger=logger)
        
        trainer.train()
        

        

Validating Guacamol dataset...


79568it [00:10, 7837.35it/s]


Guacamol dataset loaded with 79568 valid examples!
Loaded osmb data.
number of parameters: 38.06M
tokens per iteration will be: 512
Using cuda device
num decayed parameter tensors: 63, with 38,115,840 parameters
num non-decayed parameter tensors: 25, with 12,800 parameters
using fused AdamW: True
Compiling model..


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Evaluation at iter 0: train loss 0.0276, val loss 0.0550, percent 0.0000
Evaluation at iter 1: train loss 0.0747, val loss 0.0538, percent 0.0000
Evaluation at iter 2: train loss 0.0438, val loss 0.0227, percent 0.0000
Training finished.0, time 2473.47ms.
number of parameters: 38.06M
tokens per iteration will be: 512
Using cuda device
num decayed parameter tensors: 63, with 38,115,840 parameters
num non-decayed parameter tensors: 25, with 12,800 parameters
using fused AdamW: True
Compiling model..


...

Evaluation at iter 0: train loss 1.9106, val loss 1.5000, percent 0.0000
Evaluation at iter 1: train loss 1.5707, val loss 1.6001, percent 0.0000
Evaluation at iter 2: train loss 1.3970, val loss 1.4243, percent 0.0000
Training finished.8, time 2454.09ms.
number of parameters: 38.06M
tokens per iteration will be: 512
Using cuda device
num decayed parameter tensors: 63, with 38,115,840 parameters
num non-decayed parameter tensors: 25, with 12,800 parameters
using fused AdamW: True
Compiling model..


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


iter,▁▅█
lr,▁▅█
train/loss,█▃▁
train/loss_supervised,█▅▁
train/loss_unsupervised,█▄▁
val/loss,▄█▁
val/loss_supervised,█▆▁
val/loss_unsupervised,█▇▁
val/valid,▁▁▁
iter,2
lr,0.0


Evaluation at iter 0: train loss 2.7217, val loss 2.3374, percent 0.0000


OutOfMemoryError: CUDA out of memory. Tried to allocate 246.00 MiB. GPU 0 has a total capacty of 1.83 GiB of which 94.94 MiB is free. Including non-PyTorch memory, this process has 1.73 GiB memory in use. Of the allocated memory 1.44 GiB is allocated by PyTorch, and 218.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
import torch

In [6]:
params = torch.nn.parameter.Parameter(torch.ones(64, 256), requires_grad=True)
params_no_weight_decay = torch.nn.parameter.Parameter(torch.ones(64, 256), requires_grad=True)
weight_decay = 0.1

In [7]:
from hybrid_transformer.utils.optimization import AdamW

In [15]:
optimizer = AdamW( 
    [{'params': params, 'weight_decay': weight_decay}, {'params': params_no_weight_decay, 'weight_decay': 0.0}], lr=0.1, betas=(0.9, 0.999), fused=True, correct_bias=True
)

In [16]:
optimizer.step()

In [17]:
optimizer.param_groups

[{'params': [Parameter containing:
   tensor([[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]], requires_grad=True)],
  'weight_decay': 0.1,
  'lr': 0.1,
  'betas': (0.9, 0.999),
  'eps': 1e-06,
  'correct_bias': True,
  'fused': True},
 {'params': [Parameter containing:
   tensor([[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]], requires_grad=True)],
  'weight_decay': 0.0,
  'lr': 0.1,
  'betas': (0.9, 0.999),
  'eps': 1e-06,
  'correct_bias': True,
  'fused': True}]

In [18]:
from hybrid_transformer.utils.optimization import AdamW

In [19]:
for group in optimizer.param_groups:
    for p in group["params"]:
        print(group['correct_bias'])

True
True


In [21]:
optimizer = torch.optim.AdamW(
    [{'params': params, 'weight_decay': weight_decay}, {'params': params_no_weight_decay, 'weight_decay': 0.0}],
    lr=0.1, betas=(0.9, 0.999), {'correct_bias':True})

SyntaxError: positional argument follows keyword argument (241197246.py, line 3)

In [27]:
import os
from urllib.request import urlretrieve
from numpy import genfromtxt


FileNotFoundError: my_file.csv not found.

In [26]:
path = './data/molecule_net'
os.makedirs(path, exist_ok=True)
urlretrieve('https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/delaney-processed.csv', os.path.join(path, 'data.csv'))

('./data/molecule_net/data.csv', <http.client.HTTPMessage at 0x7f9c40c4f0d0>)

In [36]:
import pandas as pd

df = pd.read_csv(os.path.join(path, 'data.csv'), sep=',')
print(df.values)

[['Amigdalin' -0.974 1 ... 202.32 -0.77
  'OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O ']
 ['Fenfuram' -2.885 1 ... 42.24 -3.3 'Cc1occc1C(=O)Nc2ccccc2']
 ['citral' -2.579 1 ... 17.07 -2.06 'CC(C)=CCCC(C)=CC(=O)']
 ...
 ['Thiometon' -3.323 1 ... 18.46 -3.091 'CCSCCSP(=S)(OC)OC']
 ['2-Methylbutane' -2.245 1 ... 0.0 -3.18 'CCC(C)C']
 ['Stirofos' -4.32 1 ... 44.760000000000005 -4.522
  'COP(=O)(OC)OC(=CCl)c1cc(Cl)c(Cl)cc1Cl']]


In [38]:
df['smiles']

0       OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...
1                                  Cc1occc1C(=O)Nc2ccccc2
2                                    CC(C)=CCCC(C)=CC(=O)
3                      c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
4                                                 c1ccsc1
                              ...                        
1123                                     FC(F)(F)C(Cl)Br 
1124                            CNC(=O)ON=C(SC)C(=O)N(C)C
1125                                    CCSCCSP(=S)(OC)OC
1126                                              CCC(C)C
1127                COP(=O)(OC)OC(=CCl)c1cc(Cl)c(Cl)cc1Cl
Name: smiles, Length: 1128, dtype: object

In [40]:
df['measured log solubility in mols per litre']

0      -0.770
1      -3.300
2      -2.060
3      -7.870
4      -1.330
        ...  
1123   -1.710
1124    0.106
1125   -3.091
1126   -3.180
1127   -4.522
Name: measured log solubility in mols per litre, Length: 1128, dtype: float64

In [64]:
train.X.tolist()

['CCC(C)(C)O',
 'Cc1ccccc1C',
 'CC(C)Nc1nc(Cl)nc(NC(C)C)n1',
 'CCC(C)(C)CO',
 'COC(=O)c1ccccc1',
 'CN(C)C(=S)SSC(=S)N(C)C',
 'CC1(C)CON(Cc2ccccc2Cl)C1=O',
 'O=C1CN=C(c2ccccc2)c2cc([N+](=O)[O-])ccc2N1',
 'CCC1(C)C(=O)NC(=O)NC1=O',
 'CCCCCCCCCC(=O)OC',
 'CC1(C)C(C=C(Br)Br)C1C(=O)OC(C#N)c1cccc(Oc2ccccc2)c1',
 'CCC1(CCC(C)C)C(=O)NC(=O)NC1=O',
 'Cc1c(N(C)C)c(=O)n(-c2ccccc2)n1C',
 'CCN1C(=O)c2cccnc2N(CC)c2ncccc21',
 'O=C(Nc1ccccc1)N(Cc1ccc(Cl)cc1)C1CCCC1',
 'Clc1c(Cl)c(Cl)c(-c2ccccc2)c(Cl)c1Cl',
 'NC(=S)Nc1ccccc1',
 'CCCCBr',
 'Clc1ccc(Cl)c(-c2cccc(Cl)c2Cl)c1',
 'COCOC',
 'CCOP(=S)(OCC)Oc1ccc(S(C)=O)cc1',
 'CCOC(=O)CCN(SN(C)C(=O)Oc1cccc2c1OC(C)(C)C2)C(C)C',
 'CCCCC(=O)CCCC',
 'CCCCC(=O)OCC',
 'Cc1c([N+](=O)[O-])cc([N+](=O)[O-])cc1[N+](=O)[O-]',
 'NC(=O)c1cnccn1',
 'COc1ncnc2nccnc12',
 'Oc1ccc2ccccc2c1',
 'CCOCC',
 'CNC(=O)Oc1ccccc1C1OCCO1',
 'CC(C)CCCO',
 'CCCC(=O)CC',
 'O=Cc1ccc(O)cc1',
 'C/C=C/C=O',
 'ClCCBr',
 'Oc1ccc(Cl)c(Cl)c1',
 'c1ccc2nc(-c3ccc4ccccc4n3)ccc2c1',
 'CCCCCCCCCCCCCC',
 'C

In [66]:
train.y.tolist()

[[1.5296711365579525],
 [0.11228999542893046],
 [-0.6708731435677834],
 [0.957913998272008],
 [0.5687347696908188],
 [-0.41622500634799303],
 [0.33426629617523823],
 [-0.3662563152709021],
 [0.8675859797864974],
 [-0.7957948712605111],
 [-2.5792927681659115],
 [0.1805164774764969],
 [1.2827104902730992],
 [0.08346190442291644],
 [-1.3843683959666302],
 [-2.3477071037509316],
 [0.6071722243655042],
 [0.31889131430536405],
 [-1.6510282377722596],
 [1.6882256370910296],
 [0.35252408714571387],
 [-0.8054042349291822],
 [0.21799299578431505],
 [0.37654749631739215],
 [-0.08950664161316776],
 [1.1371286306927284],
 [0.9242812254316582],
 [0.3621334508143852],
 [1.4143587725338964],
 [0.7032658610522176],
 [0.9098671799286513],
 [1.058812316793057],
 [0.9963514529466934],
 [1.6113507277416588],
 [0.8233829069106092],
 [0.8570156797509589],
 [-1.1369272814983435],
 [-2.3669258310882744],
 [0.7580392339636441],
 [1.5296711365579525],
 [-0.47868587019435693],
 [1.2702183175038264],
 [-0.80348236

In [83]:
import deepchem as dc
from deepchem.feat.molecule_featurizers.raw_featurizer import RawFeaturizer


def download_molecule_net_regression_tasks(path_to_data_dir = './data/moleculenet'):
    return None

    
def download_molecule_net_esol(path_to_data_dir = './data/moleculenet'):
    featurizer = RawFeaturizer(smiles = True)
    _, (train, val, test), _ = dc.molnet.load_delaney(featurizer=featurizer, splitter='random')
    
    X_train = train.X.tolist()
    y_train = torch.Tensor(train.y)
    
    X_val = val.X.tolist()
    y_val = torch.Tensor(val.y)
    
    X_test = test.X.tolist() # smiles.txt
    y_test = torch.Tensor(test.y) # esol.pt
    
    return X_test
    

In [84]:
train = download_molecule_net_esol()

In [85]:
train

['Clc1cc(Cl)c(Cl)c(Cl)c1',
 'Nc1ccc(S(=O)(=O)c2ccc(N)cc2)cc1',
 'Clc1ccccc1Cl',
 'CCCCC(C)(C)O',
 'CCNC(=O)C(C)OC(=O)Nc1ccccc1',
 'CCOP(=S)(OCC)SCn1c(=O)oc2cc(Cl)ccc21',
 'O=C1NC(=O)C2C3C(C(O)(c4ccccc4)c4ccccn4)=CC(C3=C(c3ccccc3)c3ccccn3)C12',
 'CC=Cc1ccc(OC)cc1',
 'CCCCCCCCCCO',
 'c1ccc2c(c1)c1cccc3ccc4cccc2c4c31',
 'ClC1=C(Cl)C(Cl)(C2(Cl)C(Cl)=C(Cl)C(Cl)=C2Cl)C(Cl)=C1Cl',
 'CCCCN(C)C(=O)Nc1ccc(Cl)c(Cl)c1',
 'CCc1ccccc1',
 'CCC(C)(C)C',
 'Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1',
 'CC1CCCCC1NC(=O)Nc1ccccc1',
 'CCCCC',
 'CC(=O)CC(C)C',
 'COCCCNc1nc(NC(C)C)nc(SC)n1',
 'C=CCC1(C(C)C)C(=O)NC(=O)NC1=O',
 'Cc1cc2ccccc2c2ccc3ccccc3c12',
 'C=CCCCCCCC',
 'CC(=O)OCC(C)C',
 'Nc1cc[nH]c(=O)n1',
 'CCN1c2ncccc2C(=O)N(C)c2cccnc21',
 'COP(=S)(OC)Oc1ccc(Sc2ccc(OP(=S)(OC)OC)cc2)cc1',
 'c1ccoc1',
 'CC(C)OC(=O)C(O)(c1ccc(Cl)cc1)c1cccc(Cl)c1',
 'CC',
 'C1=CCC=CC1',
 'CC12CCC(=O)C=C1CCC1C2C(=O)CC2(C)C1CCC2(O)C(=O)CO',
 'COP(=S)(OC)Oc1ccc([N+](=O)[O-])c(C)c1',
 'COc1ccc2ccc(=O)oc2c1CC=C(C)C',
 'C#CC(C)N(C)C(

In [86]:
tokenizer(train)

NameError: name 'tokenizer' is not defined